#**Índex Socioeconòmic Territorial (IST)**

L'índex socioeconòmic territorial (IST) és un índex creat per la Generalitat que resumeix en un únic valor diverses característiques socioeconòmiques de la població. L'índex concentra la informació de situació laboral, nivell educatiu, immigració i renda de totes les persones que resideixen en cada unitat territorial, a partir de 6 indicadors sectorials.

* **Font**: Generalitat de Catalunya - IDESCAT
* **Nom**: Índex Socioeconòmic Territorial
* **URL**: https://www.idescat.cat/dades/obertes/ist
* **Periodicitat**: anual (amb tres anys de retràs)
* **Agregació geogràfica**: secció censal

De la URL, descarregar tots els datasets 'Índex socioeconòmic territorial (IST). Índex Catalunya = 100' agregats per secció censal per tots els anys disponibles.

## IST Cleaning

Aquest codi neteja tots els datasets de l'IST i els junta en un de sol (ist_sc_clean.csv).

In [2]:
# IMPORTS
import numpy as np
import pandas as pd
import seaborn as sns
from shapely import wkt
import json
import datetime
import csv
import pandas as pd

In [3]:
FROM_YEAR = 2019  # (included)
TO_YEAR = 2022  # (included)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# PATHS

ROOT = "/content/drive/Shareddrives/GESTIÓ DE PROJECTES/X. Entregable/"

FULL_MAP_CSV = ROOT + "cleaning/raw_data/full_map.csv"

IST_SC = ["cleaning/raw_data/ist_sc_2019.csv",
          "cleaning/raw_data/ist_sc_2020.csv"]

In [7]:
full_map = pd.read_csv(FULL_MAP_CSV, dtype={"CUSEC":str, "CUMUN":str, "CSEC":str, "CDIS":str, "CMUN":str, "CUDIS":str, "CUBAR":str, "CBAR":str})

In [9]:
ist_sc_list = []
for IST_SC_YEAR_I in IST_SC:
    ist_sc_list.append(pd.read_csv(ROOT+IST_SC_YEAR_I, sep=";"))

In [10]:
ist = pd.concat(ist_sc_list).drop(columns=["estat", "concepte"])
ist = ist.rename(columns={"secció censal": "CUSEC"})
ist['CUSEC'] = ist['CUSEC'].str[:5] + ist['CUSEC'].str[6:]
ist["valor"] = ist["valor"].str.replace(",", ".").astype(float)

In [11]:
display(ist.head(3))

,any,CUSEC,valor
0,2019,2500101001,93.1
1,2019,0800101001,100.9
2,2019,0800101002,92.2


Merge Map and IST to get only the needed IST rows

In [12]:
ist_full = ist.merge(full_map["CUSEC"], on="CUSEC", how="inner")

Extend years missing

In [13]:
for year in range(FROM_YEAR, TO_YEAR+1):
    if year not in ist_full["any"].unique():
        print("missing",year,"-> copying from",ist_full["any"].max())
        last_year = ist_full["any"].max()
        last_year_data = ist_full[ist_full['any'] == last_year].copy()
        last_year_data.loc[:,"any"] = last_year_data.loc[:,"any"].apply(lambda x: year)
        ist_full = pd.concat([ist_full, last_year_data], ignore_index=True)

missing 2021 -> copying from 2020
missing 2022 -> copying from 2021


## EXPORT

Exporting the dataset into /data_clean/ path

In [14]:
IST_FULL_CSV = "/cleaning/clean_data/ist_sc_clean.csv"
ist_full.to_csv(ROOT + IST_FULL_CSV, index=False)